In [1]:
import numpy as np
import pandas as pd
import os
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
import warnings
warnings.filterwarnings("ignore")

In [2]:
year = 2016
data_dir_all = "/Volumes/LaCie SSD2/Climate"
data_dir = os.path.join(data_dir_all, str(year))
data_dir

'/Volumes/LaCie SSD2/Climate/2016'

In [3]:
output_dir_all = "/Volumes/LaCie SSD2/Climate - Daily"
output_dir = os.path.join(output_dir_all, str(year))
output_dir

'/Volumes/LaCie SSD2/Climate - Daily/2016'

In [4]:
os.listdir(data_dir)

['00841199999.csv',
 '00841499999.csv',
 '00702699999.csv',
 '00841599999.csv',
 '00841899999.csv',
 '01001099999.csv',
 '01001499999.csv',
 '01002099999.csv',
 '01003099999.csv',
 '01006099999.csv',
 '01007099999.csv',
 '01009099999.csv',
 '01008099999.csv',
 '01011099999.csv',
 '01015099999.csv',
 '01010099999.csv',
 '01016099999.csv',
 '01017099999.csv',
 '01020099999.csv',
 '01023199999.csv',
 '01023099999.csv',
 '01024099999.csv',
 '01026099999.csv',
 '01027099999.csv',
 '01025099999.csv',
 '01030099999.csv',
 '01028099999.csv',
 '01033099999.csv',
 '01035099999.csv',
 '01036099999.csv',
 '01037099999.csv',
 '01042099999.csv',
 '01043099999.csv',
 '01045099999.csv',
 '01044099999.csv',
 '01047099999.csv',
 '01046099999.csv',
 '01049099999.csv',
 '01055099999.csv',
 '01057099999.csv',
 '01052099999.csv',
 '01058099999.csv',
 '01062099999.csv',
 '01065099999.csv',
 '01059099999.csv',
 '01068099999.csv',
 '01074099999.csv',
 '01076099999.csv',
 '01078099999.csv',
 '01082099999.csv',


In [5]:
def clean_wind_data(data):
    
    data["DATE_D"] = [x[:10] for x in data["DATE"]]
    data["MNT"] = [x[12 : ] for x in data["DATE"]]
    data["TIME_M"] = data["DATE_D"] + " " + data["MNT"]
    data['TIME_M'] = pd.to_datetime(data['TIME_M'], format = '%Y%m%d %H:%M:%S.%f')
    data['WND_SPEED'] = [int(x[8:12]) for x in data["WND"]]
    data['WND_SPEED_TYPE'] = [int(x[13]) for x in data["WND"]]
    data['WND_OBS_TYPE'] = [x[6] for x in data["WND"]]
        
    data_wind = data[(data['WND_SPEED_TYPE']!=9) & (data['WND_OBS_TYPE']!="9")]
    
    resample_df = data_wind.resample('1D', on='TIME_M', label = 'right').agg( # "right" stands for in 9:30-9:33, take 9:33 as label
        {'LATITUDE': ['first'], 
         'LONGITUDE': ['first'],
         'WND_SPEED': ['max', 'min', 'mean']
        })
    
    resample_df.columns = resample_df.columns.droplevel(0)
    resample_df.columns = ["LATITUDE", "LONGITUDE", "wnd_max", "wnd_min", "wnd_mean"]
    resample_df['date'] = resample_df.index.date
    resample_df = resample_df.reset_index(drop=True)
    
    return resample_df
    
    

In [6]:
def clean_prcp_data(data):
    
    data = data[data["AA1"].notna()]
    data["DATE_D"] = [x[:10] for x in data["DATE"]]
    data["MNT"] = [x[12 : ] for x in data["DATE"]]
    data["TIME_M"] = data["DATE_D"] + " " + data["MNT"]
    data['TIME_M'] = pd.to_datetime(data['TIME_M'], format = '%Y%m%d %H:%M:%S.%f')
    
    data['PRCP_HOUR'] = [int(x[0:2]) for x in data["AA1"]]
    data['PRCP_VOLUME'] = [int(x[3:7]) for x in data["AA1"]]
    data['PRCP_TYPE'] = [x[8] for x in data["AA1"]]
    data['PRCP_QUALITY'] = [x[10] for x in data["AA1"]]
    
    data_prcp = data[(data['PRCP_VOLUME']!=9999)]
    
    data_prcp["PRCP_VOLUME_HOUR"] = data['PRCP_VOLUME'] / data['PRCP_HOUR']
    
    resample_df = data_prcp.resample('1D', on='TIME_M', label = 'right').agg( # "right" stands for in 9:30-9:33, take 9:33 as label
        {'LATITUDE': ['first'], 
         'LONGITUDE': ['first'],
         'PRCP_VOLUME_HOUR': ['max', 'mean']
        })
    
    resample_df.columns = resample_df.columns.droplevel(0)
    resample_df.columns = ["LATITUDE", "LONGITUDE", "prcp_max", "prcp_mean"]
    resample_df['date'] = resample_df.index.date
    resample_df = resample_df.reset_index(drop=True)
    
    return resample_df

In [7]:
def clean_tmp_data(data):
    
    data["DATE_D"] = [x[:10] for x in data["DATE"]]
    data["MNT"] = [x[12 : ] for x in data["DATE"]]
    data["TIME_M"] = data["DATE_D"] + " " + data["MNT"]
    data['TIME_M'] = pd.to_datetime(data['TIME_M'], format = '%Y%m%d %H:%M:%S.%f')
    
    data['TMP_QUALITY'] = [int(x[6]) for x in data["TMP"]]
    data['TMP_VALUE'] = [int(x[0:5]) for x in data["TMP"]]
    
    data_prcp = data[(data['TMP_VALUE']!=9999) & (data['TMP_QUALITY']!=9)]
    
    resample_df = data_prcp.resample('1D', on='TIME_M', label = 'right').agg( # "right" stands for in 9:30-9:33, take 9:33 as label
        {'LATITUDE': ['first'], 
         'LONGITUDE': ['first'],
         'TMP_VALUE': ['max', 'min', 'mean']
        })
    
    resample_df.columns = resample_df.columns.droplevel(0)
    resample_df.columns = ["LATITUDE", "LONGITUDE", "tmp_max", "tmp_min", "tmp_mean"]
    resample_df['date'] = resample_df.index.date
    resample_df = resample_df.reset_index(drop=True)
    
    return resample_df

In [ ]:
file_name_list = os.listdir(data_dir)
file_name_list.sort()

for i in range(len(file_name_list)):
    if i % 100 == 0 :
        print(i)
        
    file_name = file_name_list[i]
    
    if file_name[0] == ".":
        continue
    
    data = pd.read_csv(os.path.join(data_dir, file_name))
    
    if len(data) < 200:
        pass
    
    else:
        
        try:
            data_wind = clean_wind_data(data)
        
            try:
                data_prcp = clean_prcp_data(data)
            except:
                data_prcp = None
        
            data_tmp = clean_tmp_data(data)
            data_climate_merged = data_wind
        
            try:
                data_climate_merged = data_climate_merged.merge(data_prcp[["date", "prcp_max", "prcp_mean"]], left_on="date", right_on="date", how="outer")
            except:
                data_climate_merged = data_climate_merged
            
            data_climate_merged = data_climate_merged.merge(data_tmp[["date", "tmp_max", "tmp_min", "tmp_mean"]], left_on="date", right_on="date", how="outer")
            
            data_climate_merged_location = data_climate_merged[["LATITUDE", "LONGITUDE"]].drop_duplicates()
        
            #location_list = []
            #state_list = []
            #country_list = []
        
            #for i in range(len(data_climate_merged_location)):
            
            #    location = geolocator.reverse(str(data_climate_merged_location["LATITUDE"][i]) + 
            #                                  ", " + str(data_climate_merged_location["LONGITUDE"][i]) )
            #    location_list.append(location.address)
            #    country_list.append(location.address.split(", ")[-1])
            #    state_list.append(location.address.split(", ")[-2])
            
            #data_climate_merged_location["STATE"] = state_list
            #data_climate_merged_location["COUNTRY"] = country_list
            #data_climate_merged = data_climate_merged.merge(data_climate_merged_location, left_on=["LATITUDE", "LONGITUDE"], right_on=["LATITUDE", "LONGITUDE"], how="left")
        
            data_climate_merged.to_csv(os.path.join(output_dir, file_name), index=False)
        
        except:
            pass

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300


In [ ]:
location.raw['address']

In [ ]:
location.address

In [ ]:
location_list

In [ ]:
data_climate_merged

In [ ]:
data["AA1"]

In [ ]:
data_tmp

In [ ]:
data_wind

In [ ]:
data["WND"]

In [ ]:
resample_df

In [ ]:
file_name

In [ ]:
data_prcp

In [ ]:
data

In [ ]:
file_name_list